In [224]:
import tensorflow as tf
import collections
from collections import namedtuple
import pickle
import parameters
import numpy as np

In [225]:
#User2Vec = namedtuple('User2Vec', ['user_id', 'sent_ids', 'neg_ids', 'optimizer', 'loss', 'normalized_U'])
User2Vec = namedtuple('User2Vec', ['user_id', 'sent_ids', 'neg_ids', 'app', 'loss', 'normalized_U'])

In [226]:
def hinge_loss(user_embeds, word_embeds, neg_sample_ids):
    pos_score = tf.matmul(user_embeds, word_embeds, transpose_b = True)
    print('pos_score: ', pos_score)
    
    user_embeds_t = tf.transpose(user_embeds)
    neg_sample_ids_t = tf.transpose(neg_sample_ids)
    
    neg_score = tf.matmul(neg_sample_ids, user_embeds_t)
    #neg_score = tf.tensordot(neg_sample_ids_t, user_embeds_t, [0,0])
    print('neg_score: ', neg_score)

    loss = tf.maximum(0.0, 1 - tf.add(pos_score,neg_score))
    
    return loss

In [256]:
def build_model(sess, graph, embed_matrix_rows, n_users, embed_matrix):
    lam = 1e-8
    with graph.as_default():
    # Ops and variables pinned to the CPU because of missing GPU implementation
        with tf.device('/cpu:0'):
            global_step = tf.Variable(0, trainable=False)
            
            # u_j
            user_id = tf.placeholder(tf.int32, shape=[1])
            print('user_ids: ', user_id)
            U = tf.Variable(tf.random_uniform([n_users, parameters.embedding_size], -1.0, 1.0))
            print('U: ', U)
            user_embed = tf.nn.embedding_lookup(U, user_id)
            #user_embed = tf.slice(U, [0, user_id], [U.get_shape()[0], 1])
#             user_embed = tf.transpose(user_embed)
            print('user_embed: ', user_embed)

            # e_i
            E = tf.Variable(embed_matrix, dtype=tf.float32)
            print('E: ', E)
            sent_ids = tf.placeholder(tf.int32, shape=None)
            print('sent_ids: ', sent_ids)
            word_embeds = tf.nn.embedding_lookup(E, sent_ids)
            print('word_embeds :', word_embeds)
            
            # e_l
            neg_ids = tf.placeholder(tf.int32, shape=None)
            neg_sample_ids = tf.nn.embedding_lookup(E, neg_ids)

            hinge_loss_1 =  hinge_loss(user_embed, word_embeds, neg_sample_ids)
            #U_regularizer = tf.nn.l2_loss(U)
            #E_regularizer = tf.nn.l2_loss(E)

            loss = tf.reduce_mean(hinge_loss_1 )#+ (lam/2) *  U_regularizer + (lam/2) *  E_regularizer)

            
        # Construct the SGD optimizer using a learning rate of 1.0.
        #optimizer = tf.train.GradientDescentOptimizer(1e-6).minimize(loss, global_step=global_step)
        
        
        #optimizer = tf.train.GradientDescentOptimizer(1e-6)
        #grads = optimizer.compute_gradients(loss)
        #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
        #app = optimizer.apply_gradients(clipped_grads)
        
        #AdamOptimizer
        
        #optimizer = tf.train.AdamOptimizer(1e-6)
        #grads = optimizer.compute_gradients(loss)
        #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
        #app = optimizer.apply_gradients(clipped_grads)
        
        #MomentumOptimizer
        optimizer = tf.train.MomentumOptimizer(1e-8,0.9)
        grads = optimizer.compute_gradients(loss)
        clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
        app = optimizer.apply_gradients(clipped_grads)
        
        
        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(U), 1, keep_dims=True))
        normalized_U = U / norm
        
        tf.global_variables_initializer().run()
        print("Initialized")

    #model = User2Vec(user_id, sent_ids, neg_ids, optimizer, loss, normalized_U)
    model = User2Vec(user_id, sent_ids, neg_ids, app, loss, normalized_U)
    
    return model

In [257]:
def train(sess, model, n_users):
    
    user_ids = np.arange(n_users)
    max_num_steps = 10000
  
    user_idx = {}
    for prev_user, train, test, neg_samples in user_train_data:
        
        try:
            user_id = user_idx[prev_user]
        except KeyError:
            user_idx[prev_user] = len(user_idx)
        print('user: ', user_idx[prev_user])
        average_loss_step = max(parameters.checkpoint_step/10, 100)
    
        average_loss = 0
        for step in range(max_num_steps):
#             print('step: ', step)
            for id in np.random.permutation(len(train)):
                
#                 print('train[id]', len(train[id]))
                
#                 print('train[id]', len(neg_samples[id]))
#                 print('neg samples: ', neg_samples)
#                 print('train: ', train)
                
                for x in train[id]:
                    assert not np.any(np.isnan(x))
                #print('train: ', train[id])
                if train[id] == []:
                    continue
                feed_dict = {model.user_id.name: [user_idx[prev_user]], model.sent_ids.name: train[id], model.neg_ids.name: neg_samples[id]}


            #_, loss_val = sess.run([model.optimizer, model.loss], feed_dict=feed_dict)
            _, loss_val = sess.run([model.app, model.loss], feed_dict=feed_dict)
            average_loss += loss_val
            
            if step % 1000 == 0:
                if step > 0:
                    average_loss /= 1000
                print("Average loss at step ", step, ": ", average_loss)
                average_loss = 0
                
    final_embeddings = model.normalized_U.eval()
    print("Train Finished")
    return final_embeddings

In [260]:
if __name__ == '__main__':
    
    # load pickled word embeddings
    # because we want the number of users which we pickled here
    embed_matrix, unigram_prob, wrd2idx, word_counter, n_users = pickle.load(open(parameters.output_pkl, 'rb'))
    print('embed matrix shape 0: ', embed_matrix.shape[0])
    print('embed matrix shape: ', embed_matrix.shape)
    print('embed matrix shape 1: ', embed_matrix.shape[1])
    print('embed matrix len: ', len(embed_matrix))
    
    user_train_data = pickle.load(open(parameters.output, 'rb'))

    graph = tf.Graph()
    with tf.Session(graph=graph) as sess:

        model = build_model(sess, graph, embed_matrix.shape[0], n_users, embed_matrix)
        
        user_embeddings = train(sess, model, n_users)
    

embed matrix shape 0:  1034
embed matrix shape:  (1034, 128)
embed matrix shape 1:  128
embed matrix len:  1034
user_ids:  Tensor("Placeholder:0", shape=(1,), dtype=int32, device=/device:CPU:0)
U:  <tf.Variable 'Variable_1:0' shape=(86, 128) dtype=float32_ref>
user_embed:  Tensor("embedding_lookup:0", shape=(1, 128), dtype=float32, device=/device:CPU:0)
E:  <tf.Variable 'Variable_2:0' shape=(1034, 128) dtype=float32_ref>
sent_ids:  Tensor("Placeholder_1:0", dtype=int32, device=/device:CPU:0)
word_embeds : Tensor("embedding_lookup_1:0", dtype=float32, device=/device:CPU:0)
pos_score:  Tensor("MatMul:0", shape=(1, ?), dtype=float32, device=/device:CPU:0)
neg_score:  Tensor("MatMul_1:0", shape=(?, 1), dtype=float32, device=/device:CPU:0)
Initialized
user:  0
Average loss at step  0 :  0.327806830406
Average loss at step  1000 :  0.326070125282
Average loss at step  2000 :  0.322534230232
Average loss at step  3000 :  0.318998102307
Average loss at step  4000 :  0.315461895943
Average loss

KeyboardInterrupt: 